In [1]:
import sys
sys.path.append('..')
import src.HamiltonianTerms as ham
from src.MatrixToPauliString import *
from src.BinaryEncodings import *
import numpy as np
import sympy as sp
import math



dim = 16
nqubits=4
hmat = ham.number_op(dim) + 0.5*np.identity(dim)

mtoqc = MatrixToPauliString(np.asarray(hmat))
mtoqc.convert(standard_encode)

In [3]:
print(sp.expand(mtoqc.pauli_strings))

8.0*I^0*I^1*I^2*I^3 - 4.0*I^0*I^1*I^2*Z^3 - 2.0*I^0*I^1*I^3*Z^2 - 1.0*I^0*I^2*I^3*Z^1 - 0.5*I^1*I^2*I^3*Z^0


In [4]:
print(mtoqc.pauli_strings_as_list())

defaultdict(<class 'complex'>, {'IIII': (8+0j), 'ZIII': (-0.5+0j), 'IZII': (-1+0j), 'IIZI': (-2+0j), 'IIIZ': (-4+0j)})


In [5]:
from qiskit.aqua.operators import I,X,Y,Z,Zero,PauliTrotterEvolution
from qiskit.circuit import Parameter

char_to_op={'I': I, 'X': X, 'Y': Y, 'Z': Z}

In [6]:
expr = 0
for pstring, coef in mtoqc.pauli_strings_as_list().items():
    term = char_to_op[pstring[0]]
    for char in pstring[1:]:
        term = term^char_to_op[char]
    expr += coef.real*term

In [7]:
evo_time = Parameter('t')
evolution_op = (evo_time*expr).exp_i()
print(evolution_op)

e^(-i*1.0*t * SummedOp([
  16.0 * IIII,
  -1.0 * ZIII,
  -2.0 * IZII,
  -4.0 * IIZI,
  -8.0 * IIIZ
]))


In [8]:
print(evolution_op @ Zero)

ComposedOp([
  e^(-i*1.0*t * SummedOp([
    16.0 * IIII,
    -1.0 * ZIII,
    -2.0 * IZII,
    -4.0 * IIZI,
    -8.0 * IIIZ
  ])),
  DictStateFn({'0000': 1})
])


In [9]:
fixed_time = evolution_op.bind_parameters({evo_time: 1})

In [10]:
print(fixed_time @ Zero)

ComposedOp([
  e^(-i*SummedOp([
    16.0 * IIII,
    -1.0 * ZIII,
    -2.0 * IZII,
    -4.0 * IIZI,
    -8.0 * IIIZ
  ])),
  DictStateFn({'0000': 1})
])


In [15]:
trotterized = PauliTrotterEvolution(trotter_mode='suzuki').convert(fixed_time)
print(trotterized)

global phase: -3.4336
     ┌────────┐┌────────┐
q_0: ┤ RZ(-8) ├┤ RZ(-8) ├
     ├────────┤├────────┤
q_1: ┤ RZ(-4) ├┤ RZ(-4) ├
     ├────────┤├────────┤
q_2: ┤ RZ(-2) ├┤ RZ(-2) ├
     ├────────┤├────────┤
q_3: ┤ RZ(-1) ├┤ RZ(-1) ├
     └────────┘└────────┘


In [12]:
from qiskit import QuantumCircuit, transpile

In [13]:
qc = QuantumCircuit(nqubits, nqubits)

In [14]:
qc.append(trotterized, [i for i in range(nqubits)])

In [ ]:
qc.draw('mpl')

In [ ]:
transpile(qc, basis_gates = ['cx', 'u1', 'u2', 'u3', 'H', 'X', 'Y', 'Z']).draw('mpl')

In [ ]:
from qiskit import (
    execute,
    Aer)
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ
from qiskit.visualization import plot_histogram

provider = IBMQ.load_account()

In [ ]:
noise_backend = provider.get_backend('ibmq_16_melbourne')
noise_model = NoiseModel.from_backend(noise_backend)

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

qc.measure([0,1,2],[0,1,2])

In [ ]:
job = execute(qc, simulator, shots=1000, noise_model=noise_model)
result=job.result()
counts = result.get_counts(qc)
plot_histogram(counts)